# KoChatGPT

데이터를 사용하여 사전 학습된 대규모 언어 모델을 fine-tuning해 텍스트 생성 모델을 구축

* requirements  
데이터(RLHF): data_kochatgpt  
LLM 모델: GPT2  
GPU: Colab  

## Data 전처리

Prompt: Prompt는 GPT 모델에 입력되는 텍스트로, 모델이 이를 기반으로 응답을 생성. 

Completion: Completion은 GPT 모델의 응답으로 생성되는 텍스트.  
오디션 정보를 기반으로 GPT 모델이 생성할 정보 요소를 completion에 포함.

Prompt와 Completion 키를 사용하여 json 파일 생성

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ECE/data.csv')
df.info()

In [ ]:
df['prompt'] = (
    '오디션 명 : ' + df['오디션 명'] + '\r\n'+
    '오디션 내용 : ' + df['상세정보']
    
)

df['completion'] = (
    '기획사 명 : ' + df['기획사 명'] + ', '+
    '분야 : ' + df['분야'] + ', ' +
    '시작 나이 : ' + df['시작 나이']+ ', ' +
    '끝 나이 : ' + df['끝 나이']+ ', ' +
    '성별 : ' + df['성별'] + ', ' +
    '접수 마감일: ' + df['접수 마감일']
)

df[['prompt', 'completion']].to_csv('/content/drive/MyDrive/ECE/0523/data2.csv', index=False)

In [ ]:
check  = pd.read_csv('/content/drive/MyDrive/ECE/data2.csv')
check

,prompt,completion
0,오디션 명 : JYP 2023년 5월 셋째주 센터오디션\r\n오디션 내용 : 202...,"기획사 명 : JYP 엔터테인먼트, 분야 : 보컬, 랩, 댄스, 연기, 모델, 시작..."
1,오디션 명 : 2023.05 YG MONTHLY AUDITION\r\n오디션 내용 ...,"기획사 명 : YG 엔터테인먼트, 분야 : 보컬,랩,댄스,기타, 시작 나이 : 20..."
2,오디션 명 : 미스틱스토리 온라인 오디션(2023년 5월)\r\n오디션 내용 : 미...,"기획사 명 : 미스틱 스토리, 분야 : 보컬,연기,작사·작곡·프로듀서,기타, 시작 ..."
3,오디션 명 : JYP 2023년 5월 첫째주 센터오디션\r\n오디션 내용 : 202...,"기획사 명 : JYP 엔터테인먼트, 분야 : 보컬,랩,댄스,연기,모델, 시작 나이 ..."
4,오디션 명 : FNC 엔터테인먼트 2023년 5월 공개 오디션\r\n오디션 내용 :...,"기획사 명 : FNC 엔터테인먼트, 분야 : 보컬,랩,댄스,연기,기타, 시작 나이 ..."
...,...,...
271,오디션 명 : 쏘스뮤직 7 8월 공개오디션 격변의 전국투어(전국 일정)\r\n오디션...,"기획사 명 : 쏘스뮤직, 분야 : 보컬,랩,댄스, 시작 나이 : 20,012,002..."
272,오디션 명 : 폴라리스주니어 인스타그램 온라인 오디션\r\n오디션 내용 : 안녕하세...,"기획사 명 : 폴라리스 엔터테인먼트, 분야 : 보컬,랩,댄스,연기,기타, 시작 나이..."
273,오디션 명 : 2021 MLD TOUCH AUDITION\r\n오디션 내용 : 20...,"기획사 명 : MLD 엔터테인먼트, 분야 : 보컬,랩,댄스,연기,모델, 시작 나이 ..."
274,오디션 명 : Polaris junior 프로젝트 1번째 공개오디션\r\n오디션 내...,"기획사 명 : 폴라리스 엔터테인먼트, 분야 : 보컬,랩,댄스,연기,기타, 시작 나이..."


In [ ]:
prompt_in = '/content/drive/MyDrive/ECE/data2.csv'
prompt_out = '/content/drive/MyDrive/ECE/prompt.csv'

import csv
import random
data = []

with open(prompt_in, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row[0])

with open(prompt_out, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["prompt"])  

    for row in data[1:]:
        lines = row.split('\n')
        writer.writerow([row]) 
        for _ in range(9):
            random.shuffle(lines)
            prompt = '\n'.join(lines)
            writer.writerow([prompt])  

실행 x

In [ ]:
df['prompt'] = (
    df['prompt'] + '\r\n\r\n'+
    '기획사 명은? 모집 분야는? 나이에 대한 조건 중 시작 나이는? 나이에 대한 조건 중 끝 나이는? 성별 조건은? 접수 마감일은?' + '\r\n\r\n' +
    '질문에 대한 답 형식은 기획사 명 : , 분야 : , 시작 나이 : , 끝 나이 : , 성별 : , 접수 마감일 : yyyy-mm-dd' 
)

df['prompt'].to_csv('/content/drive/MyDrive/ECE/prompt.csv', index=False)

In [ ]:
input_file = '/content/drive/MyDrive/ECE/data2.csv'
output_file = '/content/drive/MyDrive/ECE/completion.csv'
data = []
with open(input_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append([row[1]]) 

output_data = [data[0]]
for row in data[1:]:
    for _ in range(10):
        output_data.append(row)

with open(output_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(output_data)

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/ECE/prompt.csv')
column1 = df1.iloc[:, 0]

df2 = pd.read_csv('/content/drive/MyDrive/ECE/completion.csv')
column2 = df2.iloc[:, 0]

final_df = pd.DataFrame({
    'prompt': column1,
    'completion': column2
})
final_df.to_csv('/content/drive/MyDrive/ECE/final.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ECE/final.csv')
df

,prompt,completion
0,오디션 명 : JYP 2023년 5월 셋째주 센터오디션\n오디션 내용 : 2023년...,"기획사 명 : JYP 엔터테인먼트, 분야 : 보컬, 랩, 댄스, 연기, 모델, 시작..."
1,\n지원분야: 보컬 / 랩 / 댄스 / 연기 / 모델\n\n날짜: 2023년 5월 ...,"기획사 명 : JYP 엔터테인먼트, 분야 : 보컬, 랩, 댄스, 연기, 모델, 시작..."
2,*보컬/랩/댄스/연기/모델 중 한 분야만 지원 가능\n\n오디션 명 : JYP 20...,"기획사 명 : JYP 엔터테인먼트, 분야 : 보컬, 랩, 댄스, 연기, 모델, 시작..."
3,We will be waiting for your participation!\n시간...,"기획사 명 : JYP 엔터테인먼트, 분야 : 보컬, 랩, 댄스, 연기, 모델, 시작..."
4,"Date: May 21st, 2023 (SUN)\n\n\n날짜: 2023년 5월 2...","기획사 명 : JYP 엔터테인먼트, 분야 : 보컬, 랩, 댄스, 연기, 모델, 시작..."
...,...,...
2755,\n\n7월 29일 일요일 광주 오후 4시 30분 [조이댄스 플러그인뮤직 아카데미 ...,"기획사 명 : 애스토리 엔터테인먼트, 분야 : 보컬,랩,댄스.연기,기타, 시작 나이..."
2756,\n\n\n*필수사항 메일제목 [2018 애스토리 전국오디션_이름_출생년도_성별]\...,"기획사 명 : 애스토리 엔터테인먼트, 분야 : 보컬,랩,댄스.연기,기타, 시작 나이..."
2757,7월 21일 토요일 대전 오후 1시 [대전 연기모델 학원(대전댄스보컬) 은행동 3호...,"기획사 명 : 애스토리 엔터테인먼트, 분야 : 보컬,랩,댄스.연기,기타, 시작 나이..."
2758,"(댄스음원은 휴대폰에 지참 - 아이폰7, 8 기종은 연결잭 필수 지참)\n\n\n\...","기획사 명 : 애스토리 엔터테인먼트, 분야 : 보컬,랩,댄스.연기,기타, 시작 나이..."


실행 X

In [ ]:
import csv
import random

input_file = '/content/drive/MyDrive/ECE/final.csv'
output_file = '/content/drive/MyDrive/ECE/final.csv'

data = []

with open(input_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

header = data[0]
shuffled_data = random.sample(data[1:], len(data[1:]))

with open(output_file, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)  
    writer.writerows(shuffled_data) 

In [ ]:
import pandas as pd
import json

csv_path = '/content/drive/MyDrive/ECE/final.csv'
df = pd.read_csv(csv_path)

data = []
for i in range(len(df)):
    prompt = df.iloc[i, 0]
    completion = df.iloc[i, 1]
    
    entry = {
        "prompt": prompt,
        "completion": completion
    }
    
    data.append(entry)

json_data = json.dumps(data)
parsed_data = json.loads(json_data)
formatted_data = json.dumps(parsed_data, indent=4, ensure_ascii=False)
#print(formatted_data[:3000])

with open('/content/drive/MyDrive/ECE/data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [4]:
import json
file_path = "/content/drive/MyDrive/ECE/data.json"

with open(file_path, "r") as file:
    data = json.load(file)

formatted_data = json.dumps(data, indent=4, sort_keys=False, ensure_ascii=False)
print(formatted_data[:3000])

[
    {
        "prompt": "오디션 명 : JYP 2023년 5월 셋째주 센터오디션\n오디션 내용 : 2023년 5월 셋째주 센터오디션 안내\n\n날짜: 2023년 5월 21일 일요일\n시간: 오후 1시~4시 (4시 이후 입장 절대 불가)\n대상: 2001년생~2012년생 / 성별, 국적 제한 없음\n장소: JYP Center (서울시 강동구 강동대로 205)\n지원분야: 보컬 / 랩 / 댄스 / 연기 / 모델\n참여방법: 현장 접수\n\n주의사항\n*신분증 반드시 준비(학생증, 여권, 등본 등) 없을 시 참여 제한\n*보컬/랩/댄스/연기/모델 중 한 분야만 지원 가능\n*댄스 지원자는 반드시 휴대폰 음원 지참(CD 및 USB 불가)\n*팀 지원 가능\n\n여러분의 많은 참여 기다리고 있겠습니다.\n감사합니다.\n\n[May, 2023 Second JYP Monthly Audition]\n\nDate: May 21st, 2023 (SUN)\nTime: pm1:00~pm4:00 (No entrance after pm4:00)\nQualification: Anyone born between 2001~2012 regardless of their gender and nationality\nPlace: JYP Center (205, Gangdong-daero Gangdong-gu, Seoul)\nAudition Categories: VOCAL / RAP / DANCE / ACTING / MODEL\nHow to Apply: Walk-in Registration\n\nKeep in Mind\n*You must bring your ID (ex. passport) to participate\n*You can only apply for one audition category. (VOCAL/RAP/DANCE/ACTING/MODEL)\n*Dance applicants must prepare their dance music on their phones. (NO 

## INSTALL

In [5]:
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

# for transformers, 최신버전은 에러발생
!pip install transformers==4.28.1
# for ColossalAI
!pip install colossalai==0.2.7

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 384.1 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
Looking in indexes: https://pypi.org/simple, ht

## import

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
import pandas as pd
import argparse
import json
import copy
import logging

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from dataclasses import dataclass

In [ ]:
# import
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#import torch
#import torch.nn as nn
#from torch.utils.data import Dataset
#from datasets import load_dataset
#import transformers
#from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
#from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
#import pandas as pd
#import argparse
#import copy
#import logging
#import json
#from dataclasses import dataclass, field

In [7]:
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.13.1+cu116
cuda version: 11.6
cudnn version:8302


## define argment

In [8]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_1_SFT', type=str, default='/content/drive/MyDrive/ECE/0523/data.json')
parser.add_argument('--model_name', type=str, default='gpt2')
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--output_dir', type=str, default='/content/drive/MyDrive/ECE/0523/output_1_SFT')

args = parser.parse_args(args=[])

# for test
args.model_name = 'skt/kogpt2-base-v2'  # SK GPT2, https://github.com/SKT-AI/KoGPT2
# args.model_name = 'ajoublue-gpt2-base'  # 아주대, https://github.com/HeegyuKim/language-model
args.max_epochs = 2
print(args)

Namespace(data_path_1_SFT='/content/drive/MyDrive/ECE/0523/data.json', model_name='skt/kogpt2-base-v2', max_epochs=2, train_batch_size=8, output_dir='/content/drive/MyDrive/ECE/0523/output_1_SFT')


## data load & config

In [9]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_no_input": (
        "### Instruction(명령어):\n {prompt} \n\n\n\n ###  Response(응답) \n"
    ),
}

In [13]:
import json

# JSON 파일 경로
json_file_path = "/content/drive/MyDrive/ECE/data.json"

tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=1024,    
)


with open(json_file_path, "r") as file:
    data = json.load(file)

# 가장 긴 prompt 찾기
max_length = 0
max_prompt = ""

for entry in data:
    prompt = entry["prompt"]
    prompt_length = len(prompt)
    if prompt_length > max_length:
        max_length = prompt_length
        max_prompt = prompt

print("가장 긴 Prompt의 길이:", max_length)

tokens = tokenizer.tokenize(max_prompt)
num_tokens = len(tokens)
print("최대 토큰 수 : ", num_tokens)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


가장 긴 Prompt의 길이: 1801
최대 토큰 수 :  927


In [14]:
def validate_json(file_path):
    try:
        with open(file_path, "r") as json_file:
            data = json.load(json_file)
            if not isinstance(data, list):
                print("Error: JSON 파일의 최상위 객체는 리스트 형태여야 합니다.")
                return False
            for item in data:
                if not isinstance(item, dict):
                    print("Error: JSON 파일의 각 아이템은 딕셔너리 형태여야 합니다.")
                    return False
                if "prompt" not in item or "completion" not in item:
                    print("Error: JSON 파일의 각 아이템은 'prompt'와 'completion' 키를 가져야 합니다.")
                    return False
    except FileNotFoundError:
        print("Error: JSON 파일이 존재하지 않습니다.")
        return False
    except json.JSONDecodeError:
        print("Error: JSON 파일의 형식이 올바르지 않습니다.")
        return False

    return True

json_file_path = "/content/drive/MyDrive/ECE/data.json"
is_valid = validate_json(json_file_path)

if is_valid:
    print("JSON 파일의 구성이 올바르게 확인되었습니다.")

JSON 파일의 구성이 올바르게 확인되었습니다.


## model preparation

In [16]:
## 모델 준비
model = AutoModelForCausalLM.from_pretrained(args.model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=512,    
)
tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    }
)    
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


## Prepare data

source and target 생성  
prompt_no_input 이용하여 source 생성  
pattern_output 이용하여 target 생성  
  
tokenization  
소스와 타겟을 토큰화하여 모델 입력 형식에 맞게 변환.  
_tokenize_fn 메서드를 사용  
토큰화된 소스는 sources_tokenized에 저장되고,  
토큰화된 소스와 타겟을 합친 예제는 examples_tokenized에 저장  
  
입력과 레이블 생성  
소스와 타겟을 이용하여 입력(input)과 레이블(labels)을 생성  
input : 토큰화된 예제에서 소스만 추출  
labels : 토큰화된 예제 전체를 복사한 후 소스의 길이만큼 -100 값으로 채움  
  
dataset 구성 입력(input_ids)과 레이블(labels)을 데이터셋 객체에 저장  

In [17]:
## prepare data
from typing import Optional, Dict, Sequence
    
class SFT_dataset(Dataset):
    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")
        
        ## format
        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # output

        ## load dataset
        data_path_1_SFT = '/content/drive/MyDrive/ECE/data.json'
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)
            if verbose:
                print('## data check ##')
                print((list_data_dict[0]))     

        ## 데이터셋 만들기, source와 target
        prompt_no_input = PROMPT_DICT["prompt_no_input"]  # 템플릿 가져오기

        # 입력
        sources = []
        for example in list_data_dict:
          tmp = prompt_no_input.format_map(example)
          sources.append(tmp)

        # 출력
        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")

        if verbose:
            idx = 0
            print((sources[idx]))
            print((targets[idx]))

        examples = [s + t for s, t in zip(sources, targets)]

        # source data tokenized
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source만
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target


        ## 입력은 source, 출력은 source+target 이지만 학습은 target 부분만
        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = IGNORE_INDEX  # source 부분은 -100으로 채운다

        data_dict = dict(input_ids=input_ids, labels=labels)        
        
        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))    
        
    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )        
        
        
    def __len__(self):
        return len(self.input_ids)

    
    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

  
train_dataset = SFT_dataset(data_path_1_SFT=args.data_path_1_SFT, tokenizer=tokenizer)
eval_dataset  = None  
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [18]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 19.3 MB/s eta 0:00:00


In [19]:
def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

## RUN

In [20]:
## 학습
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/ECE/test", 
    overwrite_output_dir=True, 
    num_train_epochs=5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    eval_steps=1,
    save_steps=500, 
    warmup_steps=1, 
    save_total_limit=0,
    prediction_loss_only=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #save_strategy='no',
)

trainer.train()
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=args.output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.211600
1000,0.026500
1500,0.007500
2000,0.001200
2500,0.000500


In [21]:
## 추론 테스트
generator = pipeline('text-generation', model=args.output_dir, tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, 
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

list_prompt = ["오디션 명 : JYP 2023년 5월 셋째주 센터오디션\n상세정보 : 2023년 5월 셋째주 센터오디션 안내\n\n접수 마감일 : 2023년 5월 21일 일요일\n시간: 오후 1시~4시 (4시 이후 입장 절대 불가)\n대상: 2001년생~2012년생 / 성별, 국적 제한 없음\n장소: JYP Center (서울시 강동구 강동대로 205)\n지원분야: 보컬 / 랩 / 댄스 / 연기 / 모델\n참여방법: 현장 접수\n\n주의사항\n*신분증 반드시 준비(학생증, 여권, 등본 등) 없을 시 참여 제한\n*보컬/랩/댄스/연기/모델 중 한 분야만 지원 가능\n*댄스 지원자는 반드시 휴대폰 음원 지참(CD 및 USB 불가)\n*팀 지원 가능\n\n여러분의 많은 참여 기다리고 있겠습니다.\n감사합니다.\n\n[May, 2023 Second JYP Monthly Audition]\n\nDate: May 21st, 2023 (SUN)\nTime: pm1:00~pm4:00 (No entrance after pm4:00)\nQualification: Anyone born between 2001~2012 regardless of their gender and nationality\nPlace: JYP Center (205, Gangdong-daero Gangdong-gu, Seoul)\nAudition Categories: VOCAL / RAP / DANCE / ACTING / MODEL\nHow to Apply: Walk-in Registration\n\nKeep in Mind\n*You must bring your ID (ex. passport) to participate\n*You can only apply for one audition category. (VOCAL/RAP/DANCE/ACTING/MODEL)\n*Dance applicants must prepare their dance music on their phones. (NO physical CD or USB)\n*You may apply as a team\n\nWe will be waiting for your participation!\nThank you.\n\n기획사 명은? 모집 분야는? 나이에 대한 조건 중 시작 나이는? 나이에 대한 조건 중 끝 나이는? 성별 조건은? 접수 마감일은?\r\n\r\n질문에 대한 답 형식은 기획사 명 : , 분야 : , 시작 나이 : , 끝 나이 : , 성별 : , 접수 마감일 : yyyy-mm-dd로 알려줘"]
list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print(('#'*70))
    print(('completion: %s'%(result[0]['generated_text'])))

Token indices sequence length is longer than the specified maximum sequence length for this model (635 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


######################################################################
completion: ### Instruction(명령어):
 오디션 명 : JYP 2023년 5월 셋째주 센터오디션
상세정보 : 2023년 5월 셋째주 센터오디션 안내

접수 마감일 : 2023년 5월 21일 일요일
시간: 오후 1시~4시 (4시 이후 입장 절대 불가)
대상: 2001년생~2012년생 / 성별, 국적 제한 없음
장소: JYP Center (서울시 강동구 강동대로 205)
지원분야: 보컬 / 랩 / 댄스 / 연기 / 모델
참여방법: 현장 접수

주의사항
*신분증 반드시 준비(학생증, 여권, 등본 등) 없을 시 참여 제한
*보컬/랩/댄스/연기/모델 중 한 분야만 지원 가능
*댄스 지원자는 반드시 휴대폰 음원 지참(CD 및 USB 불가)
*팀 지원 가능

여러분의 많은 참여 기다리고 있겠습니다.
감사합니다.

[May, 2023 Second JYP Monthly Audition]

Date: May 21st, 2023 (SUN)
Time: pm1:00~pm4:00 (No entrance after pm4:00)
Qualification: Anyone born between 2001~2012 regardless of their gender and nationality
Place: JYP Center (205, Gangdong-daero Gangdong-gu, Seoul)
Audition Categories: VOCAL / RAP / DANCE / ACTING / MODEL
How to Apply: Walk-in Registration

Keep in Mind
*You must bring your ID (ex. passport) to participate
*You can only apply for one audition category. (VOCAL/RAP/DANCE/ACTING/MODEL)
*Dance applicants m